# Evaluation  for UCF-101

In [ ]:
%matplotlib inline

import os
import sys
sys.path.append(os.path.expanduser("~/libs"))

import cv2
import numpy as np
import tensorflow as tf
import tensortools as tt

from model.frame_prediction2 import LSTMConv2DPredictionModel

### Hyperparams

In [ ]:
# Data
INPUT_SEQ_LENGTH = 10
OUTPUT_SEQ_LENGTH = 10

# Evaluation
EVAL_BATCH_SIZE = 50

In [ ]:
# validation while training
OUT_DIR_NAME = "out-eval"
NUM_SAMPLES = 10
GIF_FPS = 5

#### Directory Paths:

In [ ]:
ROOT_DIR = "/work/sauterme/"
DATA_DIR = ROOT_DIR + "data"

In [ ]:
TRAIN_DIR = ROOT_DIR + "train/ucf/ss/2l3i5hp/c326464k533s121bn/wd1e-05/mae/triplet/YN"

assert os.path.exists(TRAIN_DIR)

### Data

In [ ]:
dataset_valid = tt.datasets.ucf101.UCF101ValidDataset(DATA_DIR,
                                                      input_seq_length=INPUT_SEQ_LENGTH,
                                                      target_seq_length=OUTPUT_SEQ_LENGTH,
                                                      image_scale_factor=0.5, double_with_flipped=True,
                                                      crop_size=[32, 32], repetitions_per_epoche=8,
                                                      skip_less_movement=True)
dataset_test = tt.datasets.ucf101.UCF101TestDataset(DATA_DIR,
                                                    input_seq_length=INPUT_SEQ_LENGTH,
                                                    target_seq_length=OUTPUT_SEQ_LENGTH,
                                                    image_scale_factor=0.5, double_with_flipped=True,
                                                    crop_size=[32, 32], repetitions_per_epoche=8,
                                                    skip_less_movement=True)

### Runtime

In [ ]:
GPU_ID = 0

In [ ]:
runtime = tt.core.DefaultRuntime(train_dir=TRAIN_DIR, gpu_devices=[GPU_ID])

In [ ]:
runtime.list_checkpoints()

In [ ]:
CHECKPOINT = tt.core.LATEST_CHECKPOINT

In [ ]:
runtime.register_datasets(None, dataset_valid, dataset_test)

In [ ]:
runtime.register_model(LSTMConv2DPredictionModel(main_loss=tt.loss.bce))

Consider to restore the **EMA variables** as well when building the model. These might generate worse results in models using batch-normalization, since the shaddow variables might get restored properly...

In [ ]:
runtime.build(restore_checkpoint=CHECKPOINT, restore_model_params=True,
              restore_ema_variables=False, eval_mode=True, verbose=True)

## Evaluation

In [ ]:
runtime.validate(EVAL_BATCH_SIZE)

In [ ]:
runtime.test(EVAL_BATCH_SIZE)

## Visualizations
On which dataset we want to test on on the next section.

In [ ]:
dataset_eval = dataset_test

### Random prediction
Either as **binary** (like in training) or **float** (as in raw dataset)...

In [ ]:
def write_animation(dir_path, inputs, targets, predictions, fps):
    concat_tgt = np.concatenate((inputs, targets))
    concat_pred = np.concatenate((inputs, predictions))

    tt.utils.video.write_multi_gif(os.path.join(dir_path, "anim-{:02d}.gif".format(i)),
                                   [concat_tgt, concat_pred],
                                   fps=fps, pad_value=1.0)

    tt.utils.video.write_multi_image_sequence(os.path.join(dir_path, "timeline-{:02d}.png".format(i)),
                                              [concat_tgt, concat_pred],
                                              pad_value=1.0)

def show(inputs, targets, predictions, rows=2):
    tt.visualization.display_batch(inputs, ncols=5, nrows=rows, title="Inputs")
    tt.visualization.display_batch(targets, ncols=5, nrows=rows, title="Targets")
    tt.visualization.display_batch(predictions, ncols=5, nrows=rows, title="Predictions")

In [ ]:
dir_path = os.path.join(runtime.train_dir, OUT_DIR_NAME, "random")

inputs, targets = dataset_eval.get_batch(NUM_SAMPLES)

predictions = runtime.predict(inputs)

show(inputs[0], targets[0], predictions[0])
for i in range(inputs.shape[0]):
    write_animation(dir_path, inputs[i], targets[i], predictions[i], GIF_FPS)

### Specific Predictions TBD !!!
We are using the inputs used in _Gutgub: Adversarial Video Generation_ cropped out of the website. These consist of two sequences...

In [ ]:
SOURCE_PATH_FORMAT = "assets/other/ucf101/{}/32"

In [ ]:
def read_sequence(dir_path, seq_id):
    image_list = []
    for i in range(INPUT_SEQ_LENGTH+OUTPUT_SEQ_LENGTH):
        image_path = os.path.join(dir_path.format(str(seq_id)), "{:02d}.png".format(i))
        image = tt.utils.image.read(image_path)
        image_list.append(image)
    seq = np.array(image_list)
    seq = seq / 255.0
    seq = np.expand_dims(seq, 0)
    return seq[:,:INPUT_SEQ_LENGTH] , seq[:,INPUT_SEQ_LENGTH:] 

In [ ]:
dir_path = os.path.join(runtime.train_dir, OUT_DIR_NAME, "spec")

for i in range(2):
    inputs, targets = read_sequence(SOURCE_PATH_FORMAT, i)
    predictions = runtime.predict(inputs)
    
    show(inputs[0], targets[0], predictions[0])
    write_animation(dir_path, inputs[0], targets[0], predictions[0], GIF_FPS)

### Fulll Image
Is the FCN model worth it? :) 

In [ ]:
dataset_valid = tt.datasets.ucf101.UCF101ValidDataset(DATA_DIR,
                                                      input_seq_length=INPUT_SEQ_LENGTH,
                                                      target_seq_length=OUTPUT_SEQ_LENGTH,
                                                      image_scale_factor=0.5, double_with_flipped=False,
                                                      crop_size=[32, 32], repetitions_per_epoche=8,
                                                      skip_less_movement=False)
dataset_test = tt.datasets.ucf101.UCF101TestDataset(DATA_DIR,
                                                    input_seq_length=INPUT_SEQ_LENGTH,
                                                    target_seq_length=OUTPUT_SEQ_LENGTH,
                                                    image_scale_factor=0.5, double_with_flipped=False,
                                                    crop_size=[32, 32], repetitions_per_epoche=8,
                                                    skip_less_movement=False)

In [ ]:
dataset_eval = dataset_test

In [ ]:
runtime.register_datasets(None, dataset_valid, dataset_test)
runtime.build(restore_checkpoint=CHECKPOINT, restore_model_params=True,
              restore_ema_variables=False, eval_mode=True)

In [ ]:
dir_path = os.path.join(runtime.train_dir, OUT_DIR_NAME, "full")

inputs, targets = dataset_eval.get_batch(NUM_SAMPLES)

predictions = runtime.predict(inputs)

show(inputs[0], targets[0], predictions[0])
for i in range(inputs.shape[0]):
    write_animation(dir_path, inputs[i], targets[i], predictions[i], GIF_FPS)

### Full Image - Longer Time Range

In [ ]:
# Data
INPUT_SEQ_LENGTH = 10
OUTPUT_SEQ_LENGTH = 20

dataset_valid = tt.datasets.ucf101.UCF101ValidDataset(DATA_DIR,
                                                      input_seq_length=INPUT_SEQ_LENGTH,
                                                      target_seq_length=OUTPUT_SEQ_LENGTH,
                                                      image_scale_factor=0.5, double_with_flipped=False,
                                                      crop_size=[32, 32], repetitions_per_epoche=8,
                                                      skip_less_movement=False)
dataset_test = tt.datasets.ucf101.UCF101TestDataset(DATA_DIR,
                                                    input_seq_length=INPUT_SEQ_LENGTH,
                                                    target_seq_length=OUTPUT_SEQ_LENGTH,
                                                    image_scale_factor=0.5, double_with_flipped=False,
                                                    crop_size=[32, 32], repetitions_per_epoche=8,
                                                    skip_less_movement=False)

In [ ]:
dataset_eval = dataset_test

In [ ]:
runtime.register_datasets(None, dataset_valid, dataset_test)
runtime.build(restore_checkpoint=CHECKPOINT, restore_model_params=True,
              restore_ema_variables=False, eval_mode=True)

In [ ]:
dir_path = os.path.join(runtime.train_dir, OUT_DIR_NAME, "full-long")

inputs, targets = dataset_eval.get_batch(NUM_SAMPLES)

predictions = runtime.predict(inputs)

show(inputs[0], targets[0], predictions[0])
for i in range(inputs.shape[0]):
    write_animation(dir_path, inputs[i], targets[i], predictions[i], GIF_FPS)